In [1]:
import VASPsol as vs
import os
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
# lets make a directory ../shape_animation_data/ if it doesn't exist
if not os.path.exists('../shape_animation_data/'):
    os.makedirs('../shape_animation_data/')


In [2]:
# lets copy the files in ../grid_search_calculations/0061tet/ to ../shape_animation_data/
fname = '0008noc'
files = os.listdir('../grid_search_calculations' + '/' + fname + '/')
for file in files:
    if os.path.isfile('../grid_search_calculations'+ '/' + fname + '/' +file):
        os.system('cp ../grid_search_calculations'+ '/' + fname + '/' + file+' ../shape_animation_data/')

In [3]:
client, cluster = vs.ef.dask_workers(32, 4, 4, burst=False)
client

/home/ericfonseca/.local/lib/python3.10/site-packages/dask_jobqueue-0.8.1-py3.10.egg/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/ericfonseca/.local/lib/python3.10/site-packages/dask_jobqueue-0.8.1-py3.10.egg/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/ericfonsec

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.13.55.17:44294/status,
Dashboard: http://10.13.55.17:44294/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.13.55.17:41869,Workers: 0
Dashboard: http://10.13.55.17:44294/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.13.55.17:44294/status,
Dashboard: http://10.13.55.17:44294/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.13.55.17:41869,Workers: 0
Dashboard: http://10.13.55.17:44294/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
NC_K_default = 0.0025
TAU_K_default = 0.000525
SIGMA_K_default = 0.6
NC_KPOINTS = np.linspace(0.001, 0.005, 20)
SIGMA_KPOINTS = np.ones((1, 20))
TAU_KPOINTS = np.ones((1, 20))
NC_KPOINTS = NC_KPOINTS*NC_K_default
TAU_KPOINTS = TAU_KPOINTS*TAU_K_default
SIGMA_KPOINTS, NC_KPOINTS, TAU_KPOINTS 

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1.]]),
 array([2.50000000e-06, 3.02631579e-06, 3.55263158e-06, 4.07894737e-06,
        4.60526316e-06, 5.13157895e-06, 5.65789474e-06, 6.18421053e-06,
        6.71052632e-06, 7.23684211e-06, 7.76315789e-06, 8.28947368e-06,
        8.81578947e-06, 9.34210526e-06, 9.86842105e-06, 1.03947368e-05,
        1.09210526e-05, 1.14473684e-05, 1.19736842e-05, 1.25000000e-05]),
 array([[0.000525, 0.000525, 0.000525, 0.000525, 0.000525, 0.000525,
         0.000525, 0.000525, 0.000525, 0.000525, 0.000525, 0.000525,
         0.000525, 0.000525, 0.000525, 0.000525, 0.000525, 0.000525,
         0.000525, 0.000525]]))

In [6]:
def calc_solv(directory, NC_K, SIGMA_K, TAU):
    try:
        return vs.calculate_solvent(directory, NSW=0, dict_of_additional_tags={'NC_K': NC_K, 
                                                                               'SIGMA_K': SIGMA_K, 
                                                                               'TAU': TAU,
                                                                               'LRHOB':'TRUE'})
    except:
        return None

In [7]:
vac_job = client.submit(vs.calculate_vacuum, '../shape_animation_data/')
from dask.distributed import progress
progress(vac_job)

VBox()

In [8]:
# submit all the solv jobs
solv_jobs = []
test_solv = client.submit(calc_solv, '../shape_animation_data/' + fname, NC_KPOINTS[0], SIGMA_KPOINTS[0,0], TAU_KPOINTS[0, 0])
progress(test_solv)

VBox()

In [9]:
# submit all the solv jobs
solv_jobs = []
for i in range(NC_KPOINTS.shape[0]):
    for j in range(SIGMA_KPOINTS.shape[0]):
        for k in range(TAU_KPOINTS.shape[1]):
            solv_jobs.append(client.submit(calc_solv, '../shape_animation_data/'+ fname, NC_KPOINTS[i], SIGMA_KPOINTS[0, j], TAU_KPOINTS[0, k]))
progress(solv_jobs)

VBox()

In [10]:
# get all the jobs on the client
all_jobs = [vac_job, test_solv,  *solv_jobs]
progress(all_jobs)

VBox()

In [ ]:
solv_jobs = client.gather(solv_jobs)
solv_df = pd.concat([i.ml_df for i in solv_jobs if i is not None])
solv_df

In [ ]:
dir = '../shape_animation_data/0008noc/'
data = vs.data(dir)
data.ml_df.sort_values(by='SIGMA_K')

In [ ]:
def load_chgcar(directory):
    try:
        return vs.Chgcar.from_file(directory + '/CHGCAR')
    except:
        return None

In [ ]:
directories = data.ml_df['directory'].values
chg_densities = [load_chgcar(i) for i in directories]
chg_densities

In [ ]:
densities = [i.get_average_along_axis(2) for i in chg_densities]

In [ ]:
for den in densities:
    plt.plot(den)
plt.ylim([0, 0.5])
plt.xlim([25, 50])